In [1]:
%%time

import os
import sys
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
import zipfile as zf

CPU times: user 518 ms, sys: 69.5 ms, total: 587 ms
Wall time: 630 ms


In [2]:
%%time

# Download the data.
url = 'http://files.grouplens.org/datasets/movielens/'


def reporthook(blocknum, blocksize, totalsize):
    readsofar = blocknum * blocksize
    if totalsize > 0:
        percent = readsofar * 1e2 / totalsize
        s = "\r%5.1f%% %*d / %d" % (
            percent, len(str(totalsize)), readsofar, totalsize)
        sys.stderr.write(s)
        if readsofar >= totalsize: # near the end
            sys.stderr.write("\n")
    else: # total size is unknown
        sys.stderr.write("read %d\n" % (readsofar,))

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename,reporthook)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename


data_file = maybe_download('ml-20m.zip', 198702078)

100.0% 198696960 / 198702078

Found and verified ml-20m.zip
CPU times: user 3.55 s, sys: 1.39 s, total: 4.94 s
Wall time: 11.7 s


100.0% 198705152 / 198702078


In [3]:
%%time

with zf.ZipFile("ml-20m.zip", "r") as zip_ref:
    zip_ref.extractall()

CPU times: user 4.12 s, sys: 706 ms, total: 4.83 s
Wall time: 4.83 s


In [4]:
%%time

# file should look like
'''
userId,movieId,rating,timestamp
1,2,3.5,1112486027
1,29,3.5,1112484676
1,32,3.5,1112484819
1,47,3.5,1112484727
1,50,3.5,1112484580
1,112,3.5,1094785740
1,151,4.0,1094785734
1,223,4.0,1112485573
1,253,4.0,1112484940
'''
m = 138_493
n = 131_262
nnz_train = 18_000_236
nnz_test = 2_000_027

data_filename = 'ml-20m/ratings.csv'

data = pd.read_csv(data_filename, dtype={0: 'int32', 1: 'int32', 2: 'float32'}, usecols=[0, 1, 2])

user = data['userId'].values
item = data['movieId'].values
rating = data['rating'].values

CPU times: user 6.38 s, sys: 748 ms, total: 7.13 s
Wall time: 5.47 s


In [5]:
%%time

print(user)
print(item)
print(rating)
print("")
print(np.min(user))
print(np.min(item))
print(np.min(rating))
print("")
print(np.max(user))
print(np.max(item))
print(np.max(rating))
print("")
print(np.unique(user).size)
print(np.unique(item).size)
print(np.unique(rating).size)
print("")
print(user.size)

assert np.max(user) == m
assert np.max(item) == n
assert user.size == nnz_train + nnz_test

[     1      1      1 ... 138493 138493 138493]
[    2    29    32 ... 69644 70286 71619]
[3.5 3.5 3.5 ... 3.  5.  2.5]

1
1
0.5

138493
131262
5.0

138493
26744
10

20000263
CPU times: user 9.46 s, sys: 321 ms, total: 9.78 s
Wall time: 2.19 s


In [6]:
%%time

user_item = np.vstack((user, item))

user_item_train, user_item_test, rating_train, rating_test = train_test_split(user_item.T,
                                                                              rating,
                                                                              test_size=nnz_test,
                                                                              random_state=42)

CPU times: user 5.56 s, sys: 578 ms, total: 6.14 s
Wall time: 2.86 s


In [7]:
%%time

#1-based to 0-based
R_test_coo = sparse.coo_matrix((rating_test, (user_item_test[:, 0] - 1, user_item_test[:, 1] - 1)))
assert R_test_coo.nnz == nnz_test

outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str(user_item_test[i, 0]) + " " + str(user_item_test[i, 1]) + " " + str(rating_test[i]) + "\n")

CPU times: user 4.95 s, sys: 77.5 ms, total: 5.02 s
Wall time: 4.96 s


In [8]:
%%time

# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 635 ms, sys: 41 ms, total: 676 ms
Wall time: 45.9 ms


In [9]:
print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[3.5 2.  3.5 ... 2.5 3.5 5. ]
[122269  49017  89526 ...  29334 124294  73173]
[  8359     31 109373 ...   3615  53124     47]

[3.5 2.  3.5 ... 2.5 3.5 5. ]
[122269  49017  89526 ...  29334 124294  73173]
[  8359     31 109373 ...   3615  53124     47]


In [10]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_test_coo.col).size)

5.0
138492
131257

0.5
0
0

138493
135697
26744
17719
CPU times: user 9.05 s, sys: 271 ms, total: 9.32 s
Wall time: 1.55 s


In [11]:
%%time

#1-based to 0-based
R_train_coo = sparse.coo_matrix((rating_train, (user_item_train[:, 0] - 1, user_item_train[:, 1] - 1)))
assert R_train_coo.nnz == nnz_train

outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str(user_item_train[i, 0]) + " " + str(user_item_train[i, 1]) + " " + str(rating_train[i]) + "\n")

CPU times: user 46.7 s, sys: 773 ms, total: 47.5 s
Wall time: 45.7 s


In [12]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 11.2 s, sys: 1.04 s, total: 12.2 s
Wall time: 4.28 s


In [13]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 675 ms, sys: 318 ms, total: 993 ms
Wall time: 314 ms


In [14]:
print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[2.  3.5 1.5 ... 3.5 5.  3.5]
[123513  33254 101943 ...  15190  92010 111372]
[3985 6723 5503 ... 2716 1275 2639]

[2.  3.5 1.5 ... 3.5 5.  3.5]
[123513  33254 101943 ...  15190  92010 111372]
[3985 6723 5503 ... 2716 1275 2639]

[3.5 3.5 3.5 ... 3.  5.  2.5]
[    1    46    49 ... 69643 70285 71618]
[       0      158      216 ... 17999818 17999891 18000236]

[3.5 3.5 3.5 ... 3.  5.  2.5]
[    1    46    49 ... 69643 70285 71618]
[       0      158      216 ... 17999818 17999891 18000236]

[4. 5. 4. ... 4. 3. 4.]
[     2      5      7 ...  79569  65408 133046]
[       0    44817    64870 ... 18000235 18000235 18000236]

[4. 5. 4. ... 4. 3. 4.]
[     2      5      7 ...  79569  65408 133046]
[       0    44817    64870 ... 18000235 18000235 18000236]


In [15]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(item).size)
print(np.unique(R_train_coo.col).size)

5.0
138492
131261

0.5
0
0

138493
138493
26744
26325
CPU times: user 12.4 s, sys: 386 ms, total: 12.7 s
Wall time: 3.49 s


In [16]:
%%time

print("writing extra meta_modified_all file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta_modified_all file
CPU times: user 1.17 ms, sys: 67 µs, total: 1.24 ms
Wall time: 744 µs


In [17]:
%%time

print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta file
CPU times: user 829 µs, sys: 0 ns, total: 829 µs
Wall time: 603 µs
